In [1]:
import pandas as pd
from gensim.parsing import preprocessing
from gensim.summarization import textcleaner
from gensim.models import word2vec
from sklearn.preprocessing import normalize
import numpy as np

In [2]:
yelp = pd.read_csv('yelp_reviews_business_pre_vec_3_4_2016.csv')

In [4]:
yelp = yelp.iloc[:, 1:]

In [118]:
yelp_for_business_dates = yelp_for_business_dates.iloc[:, 1:]

In [113]:
yelp_for_business_dates.to_csv('review_dates_:2017-01-31')

In [6]:
yelp_join = pd.DataFrame()

In [7]:
yelp_txt_str = []
for text in yelp['text']:
    text = str(text)
    yelp_txt_str.append(text)

In [8]:
yelp_join = yelp[['business_id', 'text']]

In [10]:
yelp['text'] = yelp_txt_str

In [13]:
yelp_join = yelp_join.set_index('business_id')

In [15]:
yelp_join = yelp_join.groupby('business_id').apply(np.sum)

In [18]:
def make_reviews_list(df):
    reviews_list = []
    for review in df['text']:
        reviews_list.append(review)
    return reviews_list

In [19]:
reviews_list = make_reviews_list(yelp_join)

In [20]:
reviews_list[0]

'This review is for Lauren!! Not only is she an awesome bartender at primal but we felt right at home upon meeting her! I personally think she\'s exactly what any local brewery needs! I can\'t fairly review the beer here as my husband is the beer drinker and my go to is wine! The wine selection is awesome for any of you who don\'t do beer either! Ryan (my husband) doesn\'t seem to LOVE the beers but did enjoy the centennial IPA!!also, amazing cozy atmosphere!! Perfect spot for a chill Saturday or weeknight drink!Great selection of beer and extremely friendly service!  There is ample seating and a cool selection of games.I recently made the road trip to this area to visit the breweries. I\'ve been to D9 and Ass Clown just down the road, but never made it to Primal. Boy, that was a mistake. Primal had great beer, they had freshly popped Popcorn, and they serve wine. The inside Tap room is laid out well with plenty of seating, some open room, and several nooks for more private areas. The 

In [21]:
docs = []
all_sentences = []
for i, text in enumerate(reviews_list):
    sentences = textcleaner.split_sentences(text)
    if i%1000 == 0:
        print(i)
    doc_tokens = []
    for sent in sentences:
        tokens = preprocessing.preprocess_string(sent)
        all_sentences.append(tokens)
        doc_tokens.extend(tokens)
    docs.append(doc_tokens)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000


In [22]:
cbow_model = word2vec.Word2Vec(
    all_sentences,     
    min_count=15,   # Minimum word count threshold.
    window=5,      # Number of words around target word to consider.
    sg=0,          # cbow
    sample=1e-3 ,  
    size=500,      
    hs=1           
)

In [23]:
def create_vector_list(array_shape, X):
    vector_list = []
    for review in X:
        review_vector = np.zeros_like(array_shape)
        for word in review:
            word = word.lemma_ 
            if word in cbow_model.wv:
                vector = cbow_model.wv[word]
                review_vector += vector
        norm = normalize(review_vector.reshape(1, -1))
        norm = norm.flatten()
        vector_list.append(norm)
    return vector_list

In [24]:
x = cbow_model.wv['burger']
print(len(x))
vector_list = []
X_w2v = vector_list
for review in docs:
    review_vector = np.zeros_like(x)
    for word in review:
        #tokens_word = preprocessing.preprocess_string(word)
        #word = tokens_word.lemma_ 
        if word in cbow_model.wv:
            vector = cbow_model.wv[word]
            review_vector += vector
    norm = normalize(review_vector.reshape(1, -1))
    norm = norm.flatten()
    vector_list.append(norm)

500


In [26]:
yelp_join['w2v'] = X_w2v

In [27]:
yelp_join.to_csv('w2v_after_merge_3_5')

In [29]:
df = pd.DataFrame(X_w2v)

In [30]:
len(df)

125227

In [31]:
len(yelp_join)

125227

In [32]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.188206,-0.038283,-0.001561,-0.057868,-0.000632,-0.076042,0.029634,0.058460,0.026756,-0.088966,...,-0.039240,-0.023684,0.007351,-0.000946,0.026984,0.035198,-0.017357,-0.007677,0.034954,0.036282
1,0.024509,-0.018602,0.070642,0.052995,-0.085420,0.041633,0.022008,-0.054814,0.023416,0.092991,...,0.024672,-0.036653,0.024554,-0.029597,0.044491,0.049495,-0.022766,-0.057987,-0.020372,-0.012338
2,0.012208,-0.027274,0.042529,0.027047,-0.043110,-0.053862,0.002248,0.014111,0.046564,0.040773,...,0.062360,0.009201,-0.078431,-0.040148,0.109919,-0.017709,0.010435,-0.097088,-0.041931,0.062730
3,0.039641,0.022216,-0.004785,-0.075566,0.031147,0.026233,-0.051974,0.022366,0.060769,0.023329,...,0.000921,0.032483,-0.080718,0.090594,0.000639,0.063222,-0.034102,0.020776,-0.057081,-0.006495
4,0.091070,-0.024328,-0.019922,0.067566,-0.069579,0.022806,0.002091,0.048369,-0.017015,-0.040985,...,-0.013268,-0.010765,-0.068973,-0.025637,0.076951,0.031701,-0.009085,-0.025236,0.012274,-0.014236


In [33]:
yelp_join = yelp_join.drop('business_id', 1)

ValueError: labels ['business_id'] not contained in axis

In [34]:
yelp_join = yelp_join.drop('w2v', 1)

In [35]:
yelp_join = yelp_join.reset_index()

In [36]:
yelp_join.head()

,business_id,text
0,--7zmmkVg-IMGaXbuVd0SQ,This review is for Lauren!! Not only is she an...
1,--8LPVSo5i0Oo61X01sV9A,Dr. Purcell is good and thorough. Their office...
2,--9QQLMTbFzLJ_oT-ON3Xw,I checked my son in accidently under my name. ...
3,--9e1ONYQuAa-CB_Rrw7Tw,Came To Celebrate A Milestone Birthday With My...
4,--DdmeR16TRb3LsjG0ejrQ,I was really disappointed in this event. I bou...


In [37]:
yelp_join = pd.merge(yelp_join, df, right_index=True, left_index=True, how='inner')

In [39]:
yelp_join = yelp_join.drop('text', 1)

In [41]:
yelp_join.to_csv('w2v_after_merge_3_5_2')